In [27]:
### Library Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf 


In [28]:
### Load in Stock Data
tickers = ['GOOG', 'AAPL']
start_date = '2018-01-01'

def import_stock_data(tickers, start_date):
    data = pd.DataFrame()
    if len([tickers]) == 1:
        data[tickers] = yf.download(tickers, start_date)['Adj Close']
        data = pd.DataFrame(data)
    else:
        for t in tickers:
            data[t] = yf.download(tickers, start_date)['Adj Close']
    # Reset index to make headings in the same row
    data.reset_index(inplace = True)

    data['Date'] = pd.to_datetime(data['Date'])

    return data

stock_data = import_stock_data(tickers, start_date)
stock_data

[*********************100%%**********************]  2 of 2 completed


,Date,GOOG,AAPL
0,2018-01-02,40.670975,53.250000
1,2018-01-03,40.663895,54.124001
2,2018-01-04,40.852776,54.320000
3,2018-01-05,41.317883,55.111500
4,2018-01-08,41.164433,55.347000
...,...,...,...
1579,2024-04-12,176.550003,159.190002
1580,2024-04-15,172.690002,156.330002
1581,2024-04-16,169.380005,156.000000
1582,2024-04-17,168.000000,156.880005


In [32]:
### Compute RSI Single Tickers w/ all columns
def single_rsi(data, periods):
    # Calculate price changes
    data['Price Change'] = data['GOOG'].diff()
    # Separate gains and losses
    data['Gain'] = data['Price Change'].apply(lambda x: x if x > 0 else 0)
    data['Loss'] = data['Price Change'].apply(lambda x: abs(x) if x < 0 else 0)
    # Calculate average gains and losses over the period
    data['Average Gain'] = data['Gain'].rolling(window = periods).mean()
    data['Average Loss'] = data['Loss'].rolling(window = periods).mean()
    # Calculate RS (Relative Strength)
    data['RS'] = data['Average Gain'] / data['Average Loss']
    # Calculate RSI using the formula: RSI = 100 - (100 / (1 + RS))
    data['RSI'] = 100 - (100 / (1 + data['RS']))
    
    return data

# Calculate RSI for the given DataFrame
if len(tickers) == 1:
    complete_df = single_rsi(stock_data, periods = 14)
    rsi = complete_df['RSI'].iloc[-1]
    # Print the DataFrame with RSI values
    print(complete_df)
    print('The computed RSI is: ' + str(round(rsi, 4)))

In [35]:
### Compute RSI for Multiple Tickers 
def calculate_rsi(data, periods): 
    rsi_df = pd.DataFrame(index = data.index())
    rsi_vals = []

    for t in tickers:
        # Calculate price changes
        price_change = data[t].diff()
        # Separate gains and losses
        gain = price_change.apply(lambda x: x if x > 0 else 0)
        loss = price_change.apply(lambda x: abs(x) if x < 0 else 0)
        # Calculate average gains and losses over the period
        avg_gain = gain.rolling(window = periods).mean()
        avg_loss = loss.rolling(window = periods).mean()
        # Calculate RS (Relative Strength)
        rs = avg_gain / avg_loss
        # Calculate RSI using the formula: RSI = 100 - (100 / (1 + RS))
        rsi = 100 - (100 / (1 + rs))
        # Add RSI column to the DataFrame
        rsi_df[f'RSI_{t}'] = rsi

    return rsi

get_rsi = calculate_rsi(stock_data, periods = 14)
print(get_rsi)

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
          ...    
1579    66.172629
1580    60.285966
1581    58.613796
1582    59.648451
1583    60.054167
Name: AAPL, Length: 1584, dtype: float64
